In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# import bert
# from bert import BertModelLayer
import functools

from tensorflow import keras
from tensorflow.keras import preprocessing
import tensorflow as tf
import numpy as np
import os

import tensorflow_text as tftext
import sentencepiece as spm
import sys

import json
import math
import os
import csv
import collections
import pandas as pd

from absl import app
from absl import flags
# from absl import logging
import logging
import tensorflow as tf
import tensorflow_hub as hub

sys.path.extend(["/space/users/jgeorge/git/tensorflow_experiments/tf2.3/models"])

# pylint: disable=g-import-not-at-top,redefined-outer-name,reimported
# from official.modeling import model_training_utils

from official.nlp.modeling.models import bert_classifier, bert_pretrainer
# from official.nlp.modeling.models.bert_classifier import BertClassifier
# from official.nlp.modeling.models.bert_pretrainer import BertPretrainer
# from official.nlp import bert_modeling as modeling
# from official.nlp import bert_models
from official.nlp import optimization
from official.nlp.bert import common_flags
from official.nlp.bert import input_pipeline
from official.nlp.bert import model_saving_utils
from official.utils.misc import distribution_utils
from official.utils.misc import keras_utils
from official.nlp.bert import tokenization

from official.nlp.albert import configs as albert_configs
from official.nlp.bert import run_classifier as run_classifier_bert

# from tensorflow.python.keras.engine import network

from tensorflow.python.keras.utils import tf_utils
from sklearn.model_selection import train_test_split

In [2]:
# https://stackoverflow.com/questions/13733552/logger-configuration-to-log-to-file-and-print-to-stdout
log_filepath = "/var/extra/users/jgeorge/tf2.0/input/dish/multi_logs.txt"
file_handler = logging.handlers.RotatingFileHandler(log_filepath, maxBytes=(1048576*5), backupCount=7)
# file_handler = logging.FileHandler()
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
# logging.get_absl_handler().use_absl_log_file('absl_logging', os.path.join(main_data_folder,'Smithfield/AlwaysOn/logs'))
file_handler.setFormatter(formatter)

rootLogger = logging.getLogger()
# customLogger = logging.getLogger('TestControl')
rootLogger.setLevel(logging.INFO)
# removing existing loggers
while(len(rootLogger.handlers)>0):
    rootLogger.removeHandler(rootLogger.handlers[0])

# uncomment to add logs to a file    
# rootLogger.addHandler(file_handler)

stream_handler = logging.StreamHandler(sys.stdout)
rootLogger.addHandler(stream_handler)
print('logging info',rootLogger.handlers )


logging info [<StreamHandler stdout (NOTSET)>]


In [3]:
FLAGS = flags.FLAGS    

In [4]:
print(tf.__version__)

2.3.0


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [6]:
##restricting no of gpus
gpus = tf.config.list_physical_devices('GPU')
device_to_use = gpus[3]
tf.config.experimental.set_memory_growth(device_to_use,True)
tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
print(tf.config.get_visible_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [7]:
##setting direcory for downloading tfhub modules
os.environ["TFHUB_CACHE_DIR"] = '/space/engineering/tfhub_modules'

In [8]:
dish_data_path='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/dishDataJan17.txt'
df = pd.read_csv(dish_data_path,sep='\t',header=None,names=['filename','text','granular_intent','ru_intent'])

In [9]:
df.head()

,filename,text,granular_intent,ru_intent
0,INT-sv1appis14-1504137880316-305603_4567,can you send my bill to my mail?,billing-preferences,billing-preferences
1,INT-va1appis15-1504373018548-41332,My Wally receiver has lost Satellite signal in...,comp_part_signal_loss-issue,comp_part_signal_loss-issue
2,INT-sv1appis12-1503954587819-263368,I need a payment extension so i don't get my s...,payment_extension-request,payment_extension-request
3,8ed8e9b1-10f5-438d-c409-e616c3ff9ede,how can i find my local channels. it seems i d...,channel_package-issue,channel_package-issue
4,INT-sv1appis13-1504099917638-293735,Wanted to speak with someone about my bill,representative-request,representative-request


In [10]:
df_train,df_val,y_train,y_val = train_test_split(df,df['granular_intent'],train_size=0.9,random_state=42,stratify=df['granular_intent'])

In [11]:
# main_exp_folder = '/space/users/jgeorge/projects/k/tensorflow2-question-answering/input/dish/data/jan17_2020/'
main_exp_folder = '/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/'

In [12]:
output_folder = '/var/extra/users/jgeorge/tf2.0/output/dish/models/bert_multilingual'

In [13]:
train_csv_file='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/train.txt'
eval_csv_file='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/val.txt'

df_train.to_csv(train_csv_file,sep='\t',header=False,index=False)
df_val.to_csv(eval_csv_file,sep='\t',header=False,index=False)

### Writing a file with list of intents

In [12]:
intent_file = '/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/intentlist.txt'

In [15]:
with open(intent_file,'w',encoding='utf-8') as out_f:
    for intent in sorted(df_train['granular_intent'].unique()):
        out_f.write(intent+'\n')


In [13]:
intent_list = []
with open(intent_file,'r') as inp_f:
    for intent in inp_f:
        intent_list.append(intent.strip())

In [14]:
len(df_train)

23146

In [15]:
len(df_val)

2572

In [16]:
input_meta_data = {
    'max_seq_length':128,
    'num_labels':len(intent_list),
    'train_data_size':len(df_train),
    'eval_data_size':len(df_val),
    'default_label':'other-other'
}

In [17]:
input_meta_data

{'max_seq_length': 128,
 'num_labels': 142,
 'train_data_size': 23146,
 'eval_data_size': 2572,
 'default_label': 'other-other'}

In [20]:
input_meta_path = os.path.join(main_exp_folder,'input_metadata.json')

In [22]:
#writing metadata file
with open(input_meta_path,'w',encoding='utf-8') as jf:
    jf.write(json.dumps(input_meta_data))

In [23]:
#reading metadata file
with open(input_meta_path,'r',encoding='utf-8') as jf:
#     input_meta_data = json.loads(jf.read())
    input_meta_data = json.load(jf)

## Data preprocessing & feeding into graph
We will be reading the regular text file as input & encoding the text 
using sentence piece encoder provided along with albert & then
writing it as tfrecord. Other than the sentencepiece encoder these models does not require any preprocessing like regex-replacement, lemmatization etc 

Writing as tfrecord is not necessary, but it is a more optimized file format for reading into 
tf.data api (you can even base your tf dataset on a textfile or even a python iterator)
Also it's not necessary to use tf.data apis but this is much more optimized, like preloading data into gpu memory & optimization required if you are running across systems & all

### Sentence piece encoding

In [21]:
# multilingual bert hub url
bert_hub_url = "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3"

We need to load the used sentence piece model from the hub module, 
it will be present in the assests folder of the downloaded albert model
Since we are using hub here & not directly downloading it, the path can be fetched from hub layer

In [26]:
bert_hub_loaded = hub.load(bert_hub_url)
vocab_path = bert_hub_loaded.vocab_file.asset_path.numpy().decode('utf-8')
print(f'vocab_path {vocab_path}')
# or if you are using hub.KeraLayer
# bert_model = hub.KerasLayer(bert_hub_url,trainable=True)
# sentencepiece_path = bert_model.resolved_object.sp_model_file.asset_path.numpy()

##if you have directly downloaded the model you could do this
# sentencepiece_path = os.path.join(model_dir, "assets", "30k-clean.model")


vocab_path /space/engineering/tfhub_modules/67c5e34716a3dd645ca3507fac096a07b2c8607d/assets/vocab.txt


In [28]:
bert_preprocessor_url = "https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/1"
bert_preprocessor_loaded = hub.load(bert_preprocessor_url)

Downloaded https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/1, Total size: 3.95MB
Downloaded TF-Hub Module 'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/1'.


In [44]:
max_seq_length=128
bert_config = {}
bert_config["initializer_range"] = 0.02
# we will use this for the Dropout probability between bert layer & final Dense layer
bert_config["hidden_dropout_prob"]=0.2
num_labels = input_meta_data['num_labels']

bert_preprocessor_loaded.tokenize(tf.constant("talk to an "))

In [37]:
input_text = tf.keras.layers.Input(shape=(), dtype=tf.string,name='input_text')
tokenize = hub.KerasLayer(bert_preprocessor_loaded.tokenize)
tokenized_inputs = [tokenize(input_text)] # bert can handle 2 input_texts (segments)
 

In [51]:
bert_pack_inputs = hub.KerasLayer(bert_preprocessor_loaded.bert_pack_inputs,arguments=dict(seq_length=max_seq_length))
encoder_inputs = bert_pack_inputs(tokenized_inputs)
encoder = hub.KerasLayer(bert_hub_url,trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].
bert_output = tf.keras.layers.Dropout(rate=bert_config["hidden_dropout_prob"])(pooled_output) 
initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config["initializer_range"])
output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(bert_output)
    
# model = tf.keras.Model(inputs={'input_ids':input_ids,
#                                  'input_mask':input_mask,
#                                  'segment_ids':segment_ids},
#                          outputs=output),bert_model

model = tf.keras.Model(inputs=input_text,
                         outputs=output)

In [39]:
encoder_inputs

{'input_type_ids': <tf.Tensor 'keras_layer_3/StatefulPartitionedCall:1' shape=(None, None) dtype=int32>,
 'input_mask': <tf.Tensor 'keras_layer_3/StatefulPartitionedCall:0' shape=(None, None) dtype=int32>,
 'input_word_ids': <tf.Tensor 'keras_layer_3/StatefulPartitionedCall:2' shape=(None, None) dtype=int32>}

In [42]:
def get_multilingual_bert(bert_config,num_labels,max_seq_length,model_hub_url,preprocessor_hub_url):
    bert_preprocessor_loaded = hub.load(preprocessor_hub_url)
    input_text = tf.keras.layers.Input(shape=(), dtype=tf.string,name='input_text')
    tokenize = hub.KerasLayer(bert_preprocessor_loaded.tokenize)
    tokenized_inputs = [tokenize(input_text)] # bert can handle 2 input_texts (segments)
    
    bert_pack_inputs = hub.KerasLayer(bert_preprocessor_loaded.bert_pack_inputs,arguments=dict(seq_length=max_seq_length))
    encoder_inputs = bert_pack_inputs(tokenized_inputs)
    encoder = hub.KerasLayer(model_hub_url,trainable=True)
    outputs = encoder(encoder_inputs)
    pooled_output = outputs["pooled_output"]      # [batch_size, 768].
    sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].
    bert_output = tf.keras.layers.Dropout(rate=bert_config["hidden_dropout_prob"])(pooled_output) 
    initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config["initializer_range"])
    output_dense = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(bert_output)

#     return tf.keras.Model(inputs={'input_ids':input_ids,
#                                      'input_mask':input_mask,
#                                      'segment_ids':segment_ids},
#                              outputs=output),bert_model
    return tf.keras.Model(inputs=input_text,outputs=output_dense),encoder

### Converting the data into TFRecords
For following functionalities refer the file  - models/official/nlp/data/classifier_data_lib.py       
This includes even the sentence piece tokenization part
#### Defining an object for the 1 row of text input data

In [46]:

#in our case we are doing just classification so we only need 1 text value = text_a
class InputExample(object):
    def __init__(self,uuid,text_a,text_b=None,label=None):
        """Constructs a InputExample.
        Args:
          uuid: Unique id for the example.
          text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
          text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
          label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.uuid = uuid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


#### defining an object for the 1 row of input feature

In [47]:
### in the official one there was something called is_real_example as feature, here i skipped it since it's 
#only classification
# class InputFeatures(object):
#     def __init__(self,input_ids,input_mask,
#                 segment_ids,label_id):
#         self.input_ids = input_ids
#         self.input_mask = input_mask
#         self.segment_ids = segment_ids
#         self.label_id = label_id

        
## keeping the original text itself as features & can be processed to ids
# with the required length during dataset.map function or during the model
class InputFeatures(object):
    def __init__(self,text_a,text_b,label_id):
        self.text_a = text_a
        self.text_b = text_b
        self.label_id = label_id

#### functions for converting values to train features so that it can be saved

In [48]:
main_exp_folder

'/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/'

In [52]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    print("type constant ",type(tf.constant(0)))
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(values):
    """Returns an int64_list from a bool / enum / int / uint.
      #Note that the Feature requires a list as input"""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

In [53]:
train_data_path=os.path.join(main_exp_folder,'train.tfrecords')
eval_data_path=os.path.join(main_exp_folder,'eval.tfrecords')


In [ ]:
def convert_single_example(ex_index, example, label_list, max_seq_length,sp_proto):
    ## please refer to functionality in file models/official/nlp/data/classifier_data_lib.py 
    ##this function is based on an older version & slightly different as in it's taking care of only 
    # the classification part
    
    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    
#     CLS_ID = 2
#     SEP_ID = 3
    [CLS_ID,SEP_ID]  = sp_external_tokenizer.piece_to_id(['[CLS]','[SEP]'])
    # -2 for [CLS], [SEP] 
    # we need to add these tokens in addition to the sentence words is while keeping the max sequence length limit
    # check Bert paper for this [CLS] is added at the beginning & [SEP] after 1st sentence (in our classificatin example 
    # we don't differentiate the sentences here, so [SEP] is at the end. For challenges like SQUAD 2 sentences is 
    # where i have seen people use 2 sentences , 1st question & 2nd answer )
    
    max_word_ids = max_seq_length -2
    input_ids = []
    segment_ids = []
    input_mask = []
    input_ids.append(CLS_ID)
    
    # word_ids = tfs.encode([example.text_a],model_proto=sp_proto).values[0]
    word_ids = sp_tokenizer.tokenize(example.text_a).numpy()
    
    # trimming to the max size
    word_ids = word_ids[:max_word_ids]
    input_ids.extend(word_ids)
    input_ids.append(SEP_ID)
    segment_ids = [0]*len(input_ids)
    input_mask = [1]*len(input_ids)
    if len(input_ids)<max_seq_length:
        diff = max_seq_length - len(input_ids)
        input_ids.extend([0]*diff)
        segment_ids.extend([0]*diff)
        input_mask.extend([0]*diff)
    assert(len(input_ids)==max_seq_length)
    assert(len(segment_ids)==max_seq_length)
    assert(len(input_mask)==max_seq_length)
    label_map = {label:i for i,label in enumerate(label_list)}
    label_id = label_map[example.label]
    if ex_index < 5:
        # id_to_piece requires a python int type (not numpy int)
        input_ids_new = []
        for input_id in input_ids:
            if type(input_id) == np.int32:
                input_ids_new.append(input_id.item())
            else:
                input_ids_new.append(input_id)
        input_ids = input_ids_new
                
        logging.info("*** Example ***")
        logging.info("uuid: %s", (example.uuid))
        logging.info("input_ids: %s", " ".join(map(str,input_ids)))
        logging.info("tokens: %s",
                     " ".join(sp_external_tokenizer.id_to_piece(input_ids)))
        logging.info("input_mask: %s", " ".join(map(str,input_mask)))
        logging.info("segment_ids: %s", " ".join(map(str,segment_ids)))
        logging.info("label: %s (id = %d)", example.label, label_id)

    feature = InputFeatures(input_ids,input_mask,segment_ids,label_id)
    return feature


In [ ]:
#refer file_based_convert_examples_to_features function in models/official/nlp/data/classifier_data_lib.py
def write_training_data(input_csv_file,output_file,sp_proto=None,delimiter='\t',max_seq_length=128):
    # actually writing tfrecords is not necessary, but since 
    with open(input_csv_file,'r',encoding='utf-8') as csv_file, tf.io.TFRecordWriter(output_file) as tf_record_writer:
        #tried using binary format for bytes_list, but csv_reader requires text format
#     with open(input_csv_file,'rb') as csv_file, tf.io.TFRecordWriter(output_file) as tf_record_writer:
        csv_reader = csv.reader(csv_file,delimiter=delimiter,quotechar='"')
        for i,cols in enumerate(csv_reader):
            ##skipping filename & granular tag (granular tag & final tag are the same here)
#             yield cols[1:-1]
            features = collections.OrderedDict()
            
            uuid = cols[0]
            text = cols[1]
            text_b = ""
            if(len(cols)<4):
                print('uuid ',uuid)
            intent = cols[2]
            input_example = InputExample(uuid,text,label=intent)
            #this will encode the text,label into ids 
            feature = convert_single_example(ex_index=i,example=input_example,label_list=intent_list,
                                             max_seq_length=max_seq_length,sp_proto=sp_proto)
            
            features["text_a"] = _bytes_feature(text)
            features["text_b"] = _bytes_feature(text_b)
            features["label_id"] = _int64_feature(feature.segment_ids)
            #Note making it as a list & passing it
#             features["label_id"] = _int64_feature([feature.label_id])
#             features["is_real_example"]
            tf_example = tf.train.Example(features=tf.train.Features(feature=features))
            tf_record_writer.write(tf_example.SerializeToString())



In [ ]:
write_training_data(train_csv_file,train_data_path,sp_proto=sp_model_proto,max_seq_length=max_seq_length)
write_training_data(eval_csv_file,eval_data_path,sp_proto=sp_model_proto,max_seq_length=max_seq_length)

In [54]:
train_csv_file

'/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/train.txt'

In [57]:
for line in train_dataset.take(2):
    print(line)

tf.Tensor(b'INT-sv1appis14-1504137880316-305603_1154\tPlease freeze account\tdish_pause-query\tdish_pause-query', shape=(), dtype=string)
tf.Tensor(b'f9074f6c-1831-46ee-75af-227fbab1a6a582\thow can I watch college football?\tother-other\tother-other', shape=(), dtype=string)


In [61]:
default_label = input_meta_data['default_label']

In [160]:
class LabelToId(tf.keras.layers.Layer):
#     def __init__(self,model_proto):
    def __init__(self,filepath,default_index,*args,**kwargs):
        """ 
        filepath - path to the text file with 1 label per line
        """
        super(LabelToId, self).__init__(trainable=False,dtype=tf.string,*args, **kwargs)
        self.filepath = filepath
        self.default_index=default_index
        
    def build(self,input_shape):
        # need to convert label name to label index. So keeping line no as value & whole text line as key
        # issue with using table lookup (both file based & key value tensorbased) in tf2.2 (tf2.x) - https://github.com/tensorflow/tensorflow/issues/38305
        self.table = tf.lookup.StaticHashTable(
            tf.lookup.TextFileInitializer(self.filepath,
                                          key_dtype=tf.string,key_index=tf.lookup.TextFileIndex.WHOLE_LINE,
                                     value_dtype=tf.int64,value_index=tf.lookup.TextFileIndex.LINE_NUMBER),self.default_index)
        self.built=True
        
    def call(self, input_index):
        word_ids = self.table.lookup(input_index)
        return word_ids
    
    def get_config(self):
        config = super(LabelLookup, self).get_config()
#         config.update({'filepath': self.filepath})
        return config


In [161]:
d = {l:i for i,l  in enumerate(intent_list)}
default_index = d.get(default_label)
label_to_id = LabelToId(intent_file,default_index)

In [105]:
delimiter = '\t'
temp_dataset = tf.data.experimental.CsvDataset(train_csv_file,record_defaults=[tf.constant(''),tf.constant('MISSING')],
                                               header=False,field_delim=delimiter,use_quote_delim=True,
                                               select_cols=[1,2])
# temp_dataset = temp_dataset.map(lambda line: )

In [106]:
for line in temp_dataset.take(2):
    print(line)

(<tf.Tensor: shape=(), dtype=string, numpy=b'Please freeze account'>, <tf.Tensor: shape=(), dtype=string, numpy=b'dish_pause-query'>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'how can I watch college football?'>, <tf.Tensor: shape=(), dtype=string, numpy=b'other-other'>)


In [225]:
tf.keras.backend.clear_session()

In [227]:
preprocessor_hub_url = bert_preprocessor_url
bert_preprocessor_loaded = hub.load(preprocessor_hub_url)

11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f4e1587e280> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7f4e1587e820> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop,

In [247]:
 
# input_text = tf.keras.layers.Input(shape=(), dtype=tf.string,name='input_text')
# tokenize = hub.KerasLayer(bert_preprocessor_loaded.tokenize)
# tokenized_inputs = [tokenize(input_text)]
# bert_pack_inputs = hub.KerasLayer(bert_preprocessor_loaded.bert_pack_inputs,arguments=dict(seq_length=max_seq_length))
# encoder_inputs = bert_pack_inputs(tokenized_inputs)
max_seq_length=128
@tf.function(input_signature=[tf.TensorSpec(shape=(None),dtype=tf.string)])
def encode_text(text):
# def encode_text(text, preprocessor_hub_url,max_seq_length=128 ):
    print(f'text {text}')
#     tokenized_inputs = [tokenize(input_text)]
#     bert_preprocessor_loaded = hub.load(preprocessor_hub_url)
#     input_text = tf.keras.layers.Input(shape=(), dtype=tf.string,name='input_text')
    tokenize = hub.KerasLayer(bert_preprocessor_loaded.tokenize)
    tokenized_inputs = [tokenize(text)] # bert can handle 2 input_texts (segments)
    
    bert_pack_inputs = hub.KerasLayer(bert_preprocessor_loaded.bert_pack_inputs,arguments=dict(seq_length=max_seq_length))
    return encoder_inputs

In [252]:
def encode_label(label):
    print(f'label {label}')
    return label_to_id(label)


In [255]:
# t = temp_dataset.map(lambda columns: transform_data(columns, bert_preprocessor_url, intent_list))
t = temp_dataset.batch(2).map(lambda text, label: (encode_text(text), encode_label(label)))
for line in t.take(2):
    print(line)


AttributeError: 'Tensor' object has no attribute '_datatype_enum'

In [108]:
temp_dataset.map(lambda : v1)

<MapDataset shapes: (), types: tf.string>

#### Initializing sentence piece model

In [27]:
##loading the sentence piece model into memory
sp_model_proto = tf.io.gfile.GFile(sentencepiece_path, 'rb').read()

# sp_model = sp_model_proto
# this is a tensorflow version of sentence piece tokenizer & can be used in graphs
sp_tokenizer = tftext.SentencepieceTokenizer(model=sp_model_proto)
# this is the common python package for sentence piece tokenization, 
# it currently have more functions than the tensorflow text version of sentence piece, hence we are using it for some of the steps
# Note - this can't be used in a graph
sp_external_tokenizer = spm.SentencePieceProcessor()
sp_external_tokenizer.load(model_proto=sp_model_proto)

True

### Trying out using various functions in sentence piece tokenizers

For sentence piece we can use either the       
1) tf_sentencepiece (import as tfs here)     
2) sentencepiece (imported as spm) package        
3) sentencepiece from tensorflow_text    
4) tokenization.FullSentencePieceTokenizer - a wrapper on top of 2) sentencepiece, code is available in models/official/nlp/bert/tokenization.py      

We will eventually just use tf_sentencepiece as that's the one which is currently, available to export as a graph.      
But for the 1st step of creating training data (in tf records format) either of them will do
but i'll illustrate how to use others as well

In [28]:
actual_sentence = 'give me directions to nearest restaurant randomtext 15-08-2019 year 2019'

###3rd methond
sp_tokenizer = tftext.SentencepieceTokenizer(model=sp_model_proto)
encoded_text3 = sp_tokenizer.tokenize(actual_sentence)
print('encoded_text3 ',encoded_text3, ', decoded text3 ',sp_tokenizer.detokenize(encoded_text3.numpy()),'\n')


sp_external_tokenizer = spm.SentencePieceProcessor()
sp_external_tokenizer.load(model_proto=sp_model_proto)
encoded_text2 = sp_external_tokenizer.encode_as_ids(actual_sentence)
print('encoded_text2 ',encoded_text2, ', decoded text2 ',
      sp_external_tokenizer.decode_ids(encoded_text2),'\n')


encoded_text3  tf.Tensor(
[  590    55  7876    20  6557  3284  5477 11969   357     8  3099     8
  1323  1433   159 10114], shape=(16,), dtype=int32) , decoded text3  tf.Tensor(b'give me directions to nearest restaurant randomtext 15-08-2019 year 2019', shape=(), dtype=string) 

encoded_text2  [590, 55, 7876, 20, 6557, 3284, 5477, 11969, 357, 8, 3099, 8, 1323, 1433, 159, 10114] , decoded text2  give me directions to nearest restaurant randomtext 15-08-2019 year 2019 



Note that this returns a ragged tensor

In [29]:
sp_tokenizer.tokenize(['hi this is','test run'])

<tf.RaggedTensor [[4148, 48, 25], [1289, 485]]>

In [30]:
temp = sp_tokenizer.id_to_string([0,1,2])
temp

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'<pad>', b'<unk>', b'[CLS]'], dtype=object)>

In [31]:
pieces1 = sp_external_tokenizer.encode_as_pieces(actual_sentence)
ids1 = sp_external_tokenizer.piece_to_id(pieces1)
print('pieces1 ', pieces1 , '\nids1 ',ids1,'\n')

pieces2 = sp_external_tokenizer.encode_as_pieces('give me directions to nearestrestaurant')
ids2 = sp_external_tokenizer.piece_to_id(pieces2)
pieces2_2 = sp_external_tokenizer.id_to_piece(ids2)
decoded_piece = sp_external_tokenizer.decode_pieces(pieces2)

print('pieces2 ', pieces2, '\nids2 ',ids2,'\npieces2_2 ',pieces2_2, '\ndecoded_piece ',decoded_piece )


pieces1  ['▁give', '▁me', '▁directions', '▁to', '▁nearest', '▁restaurant', '▁random', 'text', '▁15', '-', '08', '-', '20', '19', '▁year', '▁2019'] 
ids1  [590, 55, 7876, 20, 6557, 3284, 5477, 11969, 357, 8, 3099, 8, 1323, 1433, 159, 10114] 

pieces2  ['▁give', '▁me', '▁directions', '▁to', '▁nearest', 'rest', 'au', 'rant'] 
ids2  [590, 55, 7876, 20, 6557, 11466, 1346, 7874] 
pieces2_2  ['▁give', '▁me', '▁directions', '▁to', '▁nearest', 'rest', 'au', 'rant'] 
decoded_piece  give me directions to nearestrestaurant


In [32]:
#printing out 1st 100 tokens, just to get a feel of what all entries are present
for index,token in enumerate (sp_tokenizer.id_to_string([*range(100)])):
    print(index,token.numpy().decode('utf-8'))

0 <pad>
1 <unk>
2 [CLS]
3 [SEP]
4 [MASK]
5 (
6 )
7 "
8 -
9 .
10 –
11 £
12 €
13 ▁
14 ▁the
15 ,
16 ▁of
17 ▁and
18 s
19 ▁in
20 ▁to
21 ▁a
22 '
23 ▁was
24 ▁he
25 ▁is
26 ▁for
27 ▁on
28 ▁as
29 ▁with
30 ▁that
31 ▁i
32 ▁it
33 ▁his
34 ▁by
35 ▁at
36 ▁her
37 ▁from
38 t
39 ▁she
40 ▁an
41 ▁had
42 ▁you
43 d
44 ▁be
45 :
46 ▁were
47 ▁but
48 ▁this
49 i
50 ▁are
51 ▁my
52 ▁not
53 ▁one
54 ▁or
55 ▁me
56 ▁which
57 ▁have
58 a
59 ▁they
60 ?
61 ▁him
62 e
63 ▁has
64 ▁first
65 ▁all
66 ▁their
67 ▁also
68 ing
69 ed
70 ▁out
71 ▁up
72 ▁who
73 ;
74 ▁been
75 ▁after
76 ▁when
77 ▁into
78 ▁new
79 m
80 ▁there
81 ▁two
82 ▁its
83 ▁would
84 ▁over
85 ▁time
86 ▁so
87 ▁said
88 ▁about
89 ▁other
90 ▁no
91 ▁more
92 ▁can
93 y
94 ▁then
95 ▁we
96 th
97 ▁back
98 ▁what
99 re


### Converting the data into TFRecords
For following functionalities refer the file  - models/official/nlp/data/classifier_data_lib.py       
This includes even the sentence piece tokenization part
#### Defining an object for the 1 row of text input data

In [33]:

#in our case we are doing just classification so we only need 1 text value = text_a
class InputExample(object):
    def __init__(self,uuid,text_a,text_b=None,label=None):
        """Constructs a InputExample.
        Args:
          uuid: Unique id for the example.
          text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
          text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
          label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.uuid = uuid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


#### defining an object for the 1 row of input feature

In [34]:
### in the official one there was something called is_real_example as feature, here i skipped it since it's 
#only classification
class InputFeatures(object):
    def __init__(self,input_ids,input_mask,
                segment_ids,label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


#### functions for converting values to train features so that it can be saved

In [35]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    print("type constant ",type(tf.constant(0)))
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(values):
    """Returns an int64_list from a bool / enum / int / uint.
      #Note that the Feature requires a list as input"""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

In [36]:
train_data_path=os.path.join(main_exp_folder,'train.tfrecords')
eval_data_path=os.path.join(main_exp_folder,'eval.tfrecords')


#### Function which converts a single input to Features, one of the feature being the sentencepiece encoded array

In [37]:
word_ids = sp_tokenizer.tokenize("dummy sentence want to see if this works").numpy()
word_ids = word_ids[:3]
word_ids

array([26660,  5123,   259], dtype=int32)

In [38]:
word_ids.tolist()

[26660, 5123, 259]

In [39]:
sp_external_tokenizer.id_to_piece([1,2,3])

['<unk>', '[CLS]', '[SEP]']

In [40]:
sp_external_tokenizer.id_to_piece(word_ids.tolist())
# sp_external_tokenizer.id_to_piece(map(lambda x: x.item(),input_ids))
# sp_external_tokenizer.id_to_piece([x.item() for x in input_ids])


['▁dummy', '▁sentence', '▁want']

In [41]:
[CLS_ID,SEP_ID]  = sp_external_tokenizer.piece_to_id(['[CLS]','[SEP]'])

In [42]:
def convert_single_example(ex_index, example, label_list, max_seq_length,sp_proto):
    ## please refer to functionality in file models/official/nlp/data/classifier_data_lib.py 
    ##this function is based on an older version & slightly different as in it's taking care of only 
    # the classification part
    
    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    
#     CLS_ID = 2
#     SEP_ID = 3
    [CLS_ID,SEP_ID]  = sp_external_tokenizer.piece_to_id(['[CLS]','[SEP]'])
    # -2 for [CLS], [SEP] 
    # we need to add these tokens in addition to the sentence words is while keeping the max sequence length limit
    # check Bert paper for this [CLS] is added at the beginning & [SEP] after 1st sentence (in our classificatin example 
    # we don't differentiate the sentences here, so [SEP] is at the end. For challenges like SQUAD 2 sentences is 
    # where i have seen people use 2 sentences , 1st question & 2nd answer )
    
    max_word_ids = max_seq_length -2
    input_ids = []
    segment_ids = []
    input_mask = []
    input_ids.append(CLS_ID)
    
    # word_ids = tfs.encode([example.text_a],model_proto=sp_proto).values[0]
    word_ids = sp_tokenizer.tokenize(example.text_a).numpy()
    
    # trimming to the max size
    word_ids = word_ids[:max_word_ids]
    input_ids.extend(word_ids)
    input_ids.append(SEP_ID)
    segment_ids = [0]*len(input_ids)
    input_mask = [1]*len(input_ids)
    if len(input_ids)<max_seq_length:
        diff = max_seq_length - len(input_ids)
        input_ids.extend([0]*diff)
        segment_ids.extend([0]*diff)
        input_mask.extend([0]*diff)
    assert(len(input_ids)==max_seq_length)
    assert(len(segment_ids)==max_seq_length)
    assert(len(input_mask)==max_seq_length)
    label_map = {label:i for i,label in enumerate(label_list)}
    label_id = label_map[example.label]
    if ex_index < 5:
        # id_to_piece requires a python int type (not numpy int)
        input_ids_new = []
        for input_id in input_ids:
            if type(input_id) == np.int32:
                input_ids_new.append(input_id.item())
            else:
                input_ids_new.append(input_id)
        input_ids = input_ids_new
                
        logging.info("*** Example ***")
        logging.info("uuid: %s", (example.uuid))
        logging.info("input_ids: %s", " ".join(map(str,input_ids)))
        logging.info("tokens: %s",
                     " ".join(sp_external_tokenizer.id_to_piece(input_ids)))
        logging.info("input_mask: %s", " ".join(map(str,input_mask)))
        logging.info("segment_ids: %s", " ".join(map(str,segment_ids)))
        logging.info("label: %s (id = %d)", example.label, label_id)

    feature = InputFeatures(input_ids,input_mask,segment_ids,label_id)
    return feature


In [43]:
bert_config = {}

In [44]:
bert_config["initializer_range"] = 0.02
# we will use this for the Dropout probability between bert layer & final Dense layer
bert_config["hidden_dropout_prob"]=0.2

In [45]:
# max_seq_length = bert_config.max_position_embeddings
# could even keep max_seq_length as 512, but limiting to 128 for this expt
max_seq_length = 128


#### This is the code which reads data line by line converts & writes the data in tfrecord format

In [46]:
#refer file_based_convert_examples_to_features function in models/official/nlp/data/classifier_data_lib.py
def write_training_data(input_csv_file,output_file,sentencepiece_path=None,sp_proto=None,delimiter='\t',max_seq_length=128):
    ##loading the sentence piece model into memory
    if sp_proto ==None:
        sp_proto = tf.io.gfile.GFile(sentencepiece_path, 'rb').read()
   
    with open(input_csv_file,'r',encoding='utf-8') as csv_file, tf.io.TFRecordWriter(output_file) as tf_record_writer:
        #tried using binary format for bytes_list, but csv_reader requires text format
#     with open(input_csv_file,'rb') as csv_file, tf.io.TFRecordWriter(output_file) as tf_record_writer:
        csv_reader = csv.reader(csv_file,delimiter=delimiter,quotechar='"')
        for i,cols in enumerate(csv_reader):
            ##skipping filename & granular tag (granular tag & final tag are the same here)
#             yield cols[1:-1]
            features = collections.OrderedDict()
            
            uuid = cols[0]
            text = cols[1]
            if(len(cols)<4):
                print('uuid ',uuid)
            intent = cols[2]
            input_example = InputExample(uuid,text,label=intent)
            #this will encode the text,label into ids 
            feature = convert_single_example(ex_index=i,example=input_example,label_list=intent_list,
                                             max_seq_length=max_seq_length,sp_proto=sp_proto)
            features["input_ids"] = _int64_feature(feature.input_ids)
            features["input_mask"] = _int64_feature(feature.input_mask)
            features["segment_ids"] = _int64_feature(feature.segment_ids)
            #Note making it as a list & passing it
            features["label_id"] = _int64_feature([feature.label_id])
#             features["is_real_example"]
            tf_example = tf.train.Example(features=tf.train.Features(feature=features))
            tf_record_writer.write(tf_example.SerializeToString())



In [47]:
write_training_data(train_csv_file,train_data_path,sp_proto=sp_model_proto,max_seq_length=max_seq_length)
write_training_data(eval_csv_file,eval_data_path,sp_proto=sp_model_proto,max_seq_length=max_seq_length)

*** Example ***
uuid: INT-sv1appis14-1504137880316-305603_1154
input_ids: 2 13 1 413 6105 11551 2176 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
tokens: [CLS] ▁ <unk> le ase ▁freeze ▁account [SEP] <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

### Reading the data from tfrecord to provide to training graph
Before we write the actual code to read it let's just explore some of the functions in tf.data api & on how to decode the tfrecord data

In [47]:
raw_ds = tf.data.TFRecordDataset(train_data_path)

In [48]:
def _decode_data(record):
    features = {
        "input_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "input_mask":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "segment_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "label_id":tf.io.FixedLenFeature([],tf.int64)
    }
#     features = {
#         'uuid':tf.io.FixedLenFeature([],tf.string),
#         'text':tf.io.VarLenFeature(tf.int64),
#         'intent':tf.io.FixedLenFeature([],tf.string)
#     }    
    return tf.io.parse_single_example(record,features=features)


In [49]:
## map function processes line by line, similar to spark,scala map or pandas apply fucntion
processed_data = raw_ds.map(_decode_data)

In [51]:
## taking only 2 rows & iterating & viewing output
for line in processed_data.take(2):
    print(line)

{'input_ids': <tf.Tensor: shape=(128,), dtype=int64, numpy=
array([    2,    13,     1,   413,  6105, 11551,  2176,     3,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,   

#### Function for reading & creating a dataset object from tfrecord
putting the same code above into a function

In [50]:
def get_classifier_dataset(input_file,seq_length,batch_size,is_training=True):
# this is a simplified version & slightly less optimized version of what is used in official bert training
# refer to function create_classifier_dataset in models/official/nlp/data/create_finetuning_data.py


    # create a tf_data set out of the tfrecords file
    dataset = tf.data.TFRecordDataset(input_file)
    name_to_features = {
        "input_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "input_mask":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "segment_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "label_id":tf.io.FixedLenFeature([],tf.int64)
    }
    ## map function processes line by line, similar to spark,scala map or pandas apply fucntion
    dataset = dataset.map(lambda record: tf.io.parse_single_example(record, name_to_features),
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)
#     or could even do this
#     dataset = dataset.map(_decode_data)
    
#     now separating out the features & label
    def _select_data_from_record(record):
#         x contains the features
#         y is your prediction
# This dataset will be passed to keras's model.fit refer to it's documentation for further details
# a short snippet from that documentation

# A `tf.data` dataset. Should return a tuple
#         of either `(inputs, targets)` or
#         `(inputs, targets, sample_weights)`.

# Even a tuple of values for x works, but list doesn't work
#         x = (
#             record['input_ids'],
#             record['input_mask'],
#             record['segment_ids']
#         )

        x = {
            'input_ids': record['input_ids'],
            'input_mask': record['input_mask'],
            'segment_ids': record['segment_ids']
        }

        y = record['label_id']
        # our dataset is returning a tuple (x,y) - x are the features
        return (x, y)
    
    dataset = dataset.map(_select_data_from_record,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # caching to avoid overhead from reading data
    dataset = dataset.cache()
    if is_training:
        dataset = dataset.shuffle(100)
        dataset = dataset.repeat()
    
    dataset = dataset.batch(batch_size)
#     dataset = dataset.prefetch(5)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset
    

## Model code, graph creation
This section is the core logic of the model, here we are using tensorflow hub url for albert model. Using hub simplifies the code a lot      
It's a slightly simplied version of official bert code, that code have functionality to load a non hub model. 
#### refer to models/official/nlp/bert/run_classifier.py
Since in this notebook we are only using one gpu, we can even ignore the strategy part

In [51]:
# strategy = distribution_utils.get_distribution_strategy('mirrored',num_gpus=1)

# strategy = tf.distribute.OneDeviceStrategy("device:GPU:2")
#since devices to use is set to 2 already, only 1 device is visible which is 0
strategy = tf.distribute.OneDeviceStrategy("device:GPU:0")

### Temporary tryouts

In [52]:
bert_keras_layer = hub.KerasLayer(bert_hub_url,trainable=True,tags=None)

In [53]:
bert_keras_layer.non_trainable_variables

[]

In [54]:
bert_loaded = hub.load(bert_hub_url,tags=None)

In [55]:
bert_keras_layer.resolved_object.sp_model_file.asset_path.numpy()

b'/space/engineering/tfhub_modules/d0ceaf43f67b8744561ebeeaea4c7c188a6e6f78/assets/30k-clean.model'

In [56]:
bert_keras_layer.get_config()

{'name': 'keras_layer',
 'trainable': True,
 'dtype': 'float32',
 'handle': 'https://tfhub.dev/tensorflow/albert_en_large/1'}

In [57]:
temp = bert_loaded.signatures['serving_default']
temp

<ConcreteFunction signature_wrapper(input_mask, input_type_ids, input_word_ids) at 0x7F811C4DA700>

In [58]:
bert_hub_loaded.sp_model_file.asset_path

<tf.Tensor: shape=(), dtype=string, numpy=b'/space/engineering/tfhub_modules/d0ceaf43f67b8744561ebeeaea4c7c188a6e6f78/assets/30k-clean.model'>

### Defining the graph
It's just this function & your graph definition is done :)

In [59]:
def get_albert_model(bert_config,num_labels,max_seq_length,hub_url):
    """This function return our classification model based on bert + original bert model"""
#     https://www.tensorflow.org/hub/reusable_saved_models
    ## Note that name parameter specified here is the same as the feature name keys in the dictionary (x) in tf.data dataset  
    # Apparantely the code even works even if the name of the input specified here & as the key value to inputs dictionary of tf.keras.Model
    # are different from the tf.data.Dataset input. But let's not rely on that, that seems like a potential bug
    input_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_ids')
    input_mask = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_mask')
    segment_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='segment_ids')
    
    bert_model = hub.KerasLayer(hub_url,trainable=True,tags=None)
    ### pooled_output will give the representation for [CLS]
    ### sequence_output will give representations for all tokens
    ##since it's classification task we will just use pooled_output
    pooled_output, sequence_output = bert_model([input_ids, input_mask, segment_ids])
    bert_output = tf.keras.layers.Dropout(rate=bert_config["hidden_dropout_prob"])(pooled_output) 
    
    initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config["initializer_range"])
    output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(bert_output)
    
    return tf.keras.Model(inputs={'input_ids':input_ids,
                                 'input_mask':input_mask,
                                 'segment_ids':segment_ids},
                         outputs=output),bert_model

In [60]:
## clearing the existing graphs
tf.keras.backend.clear_session()

### Let's look at the classification model description/summary
We are calling get_albert_model here just to print out model summary, otherwise we call it internally
in another function

In [61]:
classification_model,core_bert_model = get_albert_model(bert_config=bert_config,
                                                              num_labels=input_meta_data['num_labels'],
                                                              max_seq_length=max_seq_length,
                                                               hub_url=bert_hub_url)

In [62]:
classification_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 17683968    input_ids[0][0]                  
                                                                 input_mask[0][0]      

In [63]:
classification_model.get_config()

{'name': 'functional_1',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 128),
    'dtype': 'int32',
    'sparse': False,
    'ragged': False,
    'name': 'input_ids'},
   'name': 'input_ids',
   'inbound_nodes': []},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 128),
    'dtype': 'int32',
    'sparse': False,
    'ragged': False,
    'name': 'input_mask'},
   'name': 'input_mask',
   'inbound_nodes': []},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 128),
    'dtype': 'int32',
    'sparse': False,
    'ragged': False,
    'name': 'segment_ids'},
   'name': 'segment_ids',
   'inbound_nodes': []},
  {'class_name': 'KerasLayer',
   'config': {'name': 'keras_layer',
    'trainable': True,
    'dtype': 'float32',
    'handle': 'https://tfhub.dev/tensorflow/albert_en_large/1'},
   'name': 'keras_layer',
   'inbound_nodes': [[['input_ids', 0, 0, {}],
     ['input_mask', 0, 0, {}],
     ['segment_ids', 

In [64]:
core_bert_model.get_config()

{'name': 'keras_layer',
 'trainable': True,
 'dtype': 'float32',
 'handle': 'https://tfhub.dev/tensorflow/albert_en_large/1'}

#### Define Loss function,  metric

In [65]:
def get_loss_fn(num_classes, loss_factor=1.0):
    """Gets the classification loss function."""

    def classification_loss_fn(labels, logits):
        """Classification loss."""
        labels = tf.squeeze(labels)
#         log_probs = tf.nn.log_softmax(logits, axis=-1)
#         one_hot_labels = tf.one_hot(
#             tf.cast(labels, dtype=tf.int32), depth=num_classes, dtype=tf.float32)
#         per_example_loss = -tf.reduce_sum(
#             tf.cast(one_hot_labels, dtype=tf.float32) * probs, axis=-1)
        per_example_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = labels, logits=logits)
        #batch loss
        loss = tf.reduce_mean(per_example_loss)
        loss *= loss_factor
        return loss
    return classification_loss_fn  


def metric_fn():
    return tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy',dtype=tf.float32)

#### Function for running training

In [66]:
def run_keras_compile_fit(model_dir,
                          strategy,
                          model_fn,
                          training_dataset,
                          evaluation_dataset,
                          loss_fn,
                          metric_fn,
                          init_checkpoint,
                          epochs,
                          steps_per_epoch,
                          steps_per_loop,
                          eval_steps,
                          custom_callbacks=None):
    """Runs BERT classifier model using Keras compile/fit API."""
    ###slightly simplied version of official bert code 
    # refer to models/official/nlp/bert/run_classifier.py -   function run_keras_compile_fit
    # if you running on a single gpu or on just cpu this strategy is not necessary
    with strategy.scope():
        #sub_model is the original bert
        classification_model, sub_model = model_fn()
        optimizer = classification_model.optimizer
        
        # this is not required for the hub version of the model, this restore method is trying to restor values,
        # from a saved bert model (since we have only provided the sub_model to checkpoint)
        # Let's saying we stopped training at some point & want to continue from that point next time,
        # here we can instead load our classification_model & init_checkpoint can be our previous saved checkpoint file
        if init_checkpoint:
            checkpoint = tf.train.Checkpoint(model=sub_model)
#             checkpoint = tf.train.Checkpoint(model=classification_model)
            checkpoint.restore(init_checkpoint).assert_existing_objects_matched()
#             checkpoint.restore(init_checkpoint).expect_partial()

        classification_model.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=[metric_fn()])
    #     ,experimental_steps_per_execution=steps_per_loop)

        summary_dir = os.path.join(model_dir, 'summaries')
        summary_callback = tf.keras.callbacks.TensorBoard(summary_dir, profile_batch='10,20')
        checkpoint_path = os.path.join(model_dir, 'checkpoint')
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            checkpoint_path, save_weights_only=True)
        early_stopping = tf.keras.callbacks.EarlyStopping()

        if custom_callbacks is not None:
            custom_callbacks += [summary_callback, checkpoint_callback, early_stopping]
        else:
            custom_callbacks = [summary_callback, checkpoint_callback, early_stopping]
#       Note that we are only passing x & not y in fit function
#       Refer to keras model.fit documentation for further details

#       If `x` is a dataset, generator,
#       or `keras.utils.Sequence` instance, `y` should
#       not be specified (since targets will be obtained from `x`).
        model_history = classification_model.fit(
            x=training_dataset,
            validation_data=evaluation_dataset,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            validation_steps=eval_steps,
            callbacks=custom_callbacks)

        return classification_model

In [67]:
output_folder

'/var/extra/users/jgeorge/tf2.0/input/dish/models/albert_en_large'

In [68]:
output_folder = '/var/extra/users/jgeorge/tf2.0/input/dish/models/albert_en_large_softmax'

### The training function call

In [69]:
## clearing the existing graphs
tf.keras.backend.clear_session()

### defining various parameters for the experiment

In [70]:
steps_per_loop = 1
learning_rate=1e-5
epochs=20

train_batch_size=16
eval_batch_size=16

train_data_size = input_meta_data['train_data_size']
steps_per_epoch = int(train_data_size / train_batch_size)
print(steps_per_epoch)

warmup_steps = int(epochs * train_data_size * 0.1 / train_batch_size)
eval_steps = int(math.ceil(input_meta_data['eval_data_size'] / eval_batch_size))

1446


### training & validation dataset object creation

In [71]:
training_dataset = get_classifier_dataset(train_data_path,max_seq_length,train_batch_size,is_training=True)
evaluation_dataset = get_classifier_dataset(eval_data_path,max_seq_length,eval_batch_size,is_training=False)

In [72]:
## taking only 2 rows & iterating & viewing output
for line in training_dataset.take(2):
    print(line)

({'input_ids': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[   2,   13,    1, ...,    0,    0,    0],
       [   2,   13,    1, ...,    0,    0,    0],
       [   2,   63, 3108, ...,    0,    0,    0],
       ...,
       [   2,   13,    1, ...,    0,    0,    0],
       [   2,   13,    1, ...,    0,    0,    0],
       [   2,   13,    1, ...,    0,    0,    0]])>, 'input_mask': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'segment_ids': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>}, <tf.Tensor: shape=(16,), dtype=int64, numpy=
array([ 95,  37,  18,  53,  89,   

In [73]:
loss_multiplier = 1
max_seq_length = input_meta_data['max_seq_length']
num_classes = input_meta_data['num_labels']
loss_fn = get_loss_fn(num_classes,loss_multiplier)

def _get_classifier_model():
    classifier_model,core_model = get_albert_model(bert_config=bert_config,
                                                              num_labels=num_classes,
                                                              max_seq_length=max_seq_length,
                                                               hub_url=bert_hub_url)
    ##This is basically Adam optimizer with weight decay after set no of warm up steps & 
    # before that an increasing learning rate from 0 to initial_lr
    # refer to models/official/nlp/optimization.py for more details
    classifier_model.optimizer = optimization.create_optimizer(init_lr=learning_rate,
                                                               num_train_steps=steps_per_epoch*epochs,
                                                               num_warmup_steps=warmup_steps)
    return classifier_model,core_model


In [74]:

trained_model = run_keras_compile_fit(model_dir=output_folder,strategy=strategy,model_fn=_get_classifier_model,
                                     training_dataset=training_dataset,evaluation_dataset=evaluation_dataset,
                                      loss_fn=loss_fn,metric_fn=metric_fn,
                                     init_checkpoint=None,
                                      epochs=epochs,
                                      steps_per_epoch=steps_per_epoch,
                                      steps_per_loop=steps_per_loop,
                                      eval_steps=eval_steps
                                     )

using Adamw optimizer
Epoch 1/20
  19/1446 [..............................] - ETA: 15:23 - loss: 5.1123 - test_accuracy: 0.0197WARNING:tensorflow:From /opt/custom/python/anaconda3/envs/tf2.3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
From /opt/custom/python/anaconda3/envs/tf2.3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1284/1446 [=========================>....] - ETA: 1:55 - loss: 4.5122 - test_accuracy: 0.0881

KeyboardInterrupt: 

### trying training for a few more epochs

## Saving the model
The model that we trained right now takes the sentence piece encoded list as input, along with input_mask list, segment_ids list
When we are deploying we want to give a text as input & get the text label as output.   
So what we are going to do is to have a graph with sentence piece encoding within the graph itself & create input_mask, segment_ids also as part of the graph for input   
For the output have a softmax & a table lookup to on labels to return the label name 

### Sentence piece tokenizaton
I'll walk through details of this Layer after we are done saving the model

In [501]:

class SentencepieceTokenization(tf.keras.layers.Layer):
    def __init__(self,model_path,max_seq_length):
        super(SentencepieceTokenization, self).__init__(trainable=False,dtype=tf.int64)
        self.model_path = model_path
        self.max_seq_length = max_seq_length
        
    def build(self,input_shape):
        self.model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
        self.sp_tokenizer =  tftext.SentencepieceTokenizer(model=self.model_proto)
        ## the output from id_to_string can be used to validate CLS_ID, SEP_ID
        self.sp_tokenizer.id_to_string([0,1,2,3])
        ## pad_id is usually 0 
#         [self.CLS_ID,self.SEP_ID,self.PAD_ID]  = self.sp_tokenizer.tokenize(['[CLS]','[SEP]','<pad>'],model_file=self.model_path)
        [self.PAD_ID, self.CLS_ID, self.SEP_ID] = [0, 2, 3]
        self.built=True
    
    def call(self, input_text):
        ##tensorflow sentence piece works while exporting to graph while, tf_text sentencepiece doesn't
        encoded_text = self.get_encoded_text(input_text,max_seq_length=self.max_seq_length)
        return encoded_text
    
    # converting this to tf.function will make all the inputs a tensor & some of the operation require max_seq_length to be
    # a regular integer. Rather than passing it as an argument we could also pick up the variable from the object
#     @tf.function
    def get_encoded_text(self,input_text_batch,max_seq_length):
        # sp_encoded is a ragged tensor 2 dimensional tensor with 1st dimension being the batch size
        sp_encoded = self.sp_tokenizer.tokenize(input_text_batch)
        # tf.shape gives dynamic shape &
        # tenorflow_variable.shape gives static shape with dynamic entries = None
#             actual_token_length = tf.shape(concat)[-1]
        # tf.shape was failing with ragged tensor, hence converting it to regular tensor for getting batch dimension
        # TODO find if there is better way to get shape
        batch_size_dynamic = tf.shape(sp_encoded.to_tensor())[0]
        batch_size = sp_encoded.shape[0]
#         print(f'sp_encoded {sp_encoded}')
        # casting down to int32 from int64 as subsequent operations required int32
        sequence_length = tf.cast(sp_encoded.row_lengths(),tf.int32)
#         trimming to max_length-2 (-2 to incorporate [CLS], [SEP])
        trimmed_max_length = max_seq_length-2
#         print(f'sequence_length {sequence_length} trimmed_max_length {trimmed_max_length}')
        # keeping the batch dimension (1st dimension) as is & trimming the 2nd one
        values_trimmed = sp_encoded[:,:trimmed_max_length]
#         print(f'values_trimmed {values_trimmed}')

        # let's say batch_size = 2 then values_to_prepend will have shape=(2, 1), similary values_to_append will have the same shape
        value_to_prepend = tf.expand_dims(tf.repeat([self.CLS_ID],repeats=batch_size_dynamic),axis=1)
        value_to_append = tf.expand_dims(tf.repeat([self.SEP_ID],repeats=batch_size_dynamic),axis=1)
        concat = tf.concat([value_to_prepend,values_trimmed,value_to_append],axis=1,name='concat_out')


        # concat is a ragged tensor & to_tensor will by default pad by 0
        padded = concat.to_tensor(shape=(None,max_seq_length))
#         segment_ids = tf.zeros(shape=tf.shape(padded),dtype=tf.int32)
#         or
        # This will create zeros with similar shape as padded tensor
        # for classification use case we only have 1 segment 
        # (for tasks which require distinguishing between 1st & 2nd sentence there will be 2 values )
        segment_ids = tf.zeros_like(padded,dtype=tf.int32,name='segment_ids')
        
        # input mask have 1 as the value for all positions with a token & zero for the remaining ones
        input_mask = tf.ones_like(concat).to_tensor(shape=(None,max_seq_length))
        return (padded,segment_ids,input_mask)
    
    
    def get_config(self):
        config = super(SentencepieceTokenization, self).get_config()
        config.update({'model_path': self.model_path})
        return config

In [465]:
sentence_piece_layer_temp = SentencepieceTokenization(sentencepiece_path,5)
sentence_piece_layer_temp.build(1)

In [466]:
sentence_piece_layer_temp.call(tf.constant(['hello talk to an agent and test new text']))

(<tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[    2, 10975,   930,    20,     3]], dtype=int32)>,
 <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>,
 <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>)

In [426]:
@tf.function
def get_temp_encoded(text):
    sp_encoded = sp_tokenizer.tokenize(text)
    return sp_encoded

In [435]:
sp_encoded = get_temp_encoded(tf.constant(['hello talk to an agent and test new text',"hello","another dummy text"]))
sp_encoded

<tf.RaggedTensor [[10975, 930, 20, 40, 2267, 17, 1289, 78, 1854], [10975], [226, 26660, 1854]]>

In [317]:
class LabelLookup(tf.keras.layers.Layer):
#     def __init__(self,model_proto):
    def __init__(self,filepath,default,*args,**kwargs):
        """ 
        filepath - path to the text file with 1 label per line
        """
        super(LabelLookup, self).__init__(trainable=False,dtype=tf.int64,*args, **kwargs)
        self.filepath = filepath
        self.default=default
        
    def build(self,input_shape):
        # need to convert label index to label name. So keeping line no as key & whole text line as label
        # issue with using table lookup (both file based & key value tensorbased) in tf2.2 (tf2.x) - https://github.com/tensorflow/tensorflow/issues/38305
        self.table = tf.lookup.StaticHashTable(
            tf.lookup.TextFileInitializer(self.filepath,
                                          key_dtype=tf.int64,key_index=tf.lookup.TextFileIndex.LINE_NUMBER,
                                     value_dtype=tf.string,value_index=tf.lookup.TextFileIndex.WHOLE_LINE),self.default)
        self.built=True
        
    def call(self, input_index):
        word_ids = self.table.lookup(input_index)
        return word_ids
    
    def get_config(self):
        config = super(LabelLookup, self).get_config()
#         config.update({'filepath': self.filepath})
        return config


In [502]:
tf.keras.backend.clear_session()

In [507]:
max_seq_length = input_meta_data['max_seq_length']
num_labels = input_meta_data['num_labels']
initial_lr = learning_rate
# default_label_index = 1
default_label=input_meta_data['default_label']
max_top_intents = 5
checkpoint_path=tf.train.latest_checkpoint(output_folder)

input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
sentence_piece_layer = SentencepieceTokenization(model_path=sentencepiece_path,max_seq_length=tf.constant(max_seq_length))
input_ids_processed,segment_ids_processed,input_mask_processed = sentence_piece_layer(input_text)


bert_model = hub.KerasLayer(bert_hub_url,trainable=True,tags=None)
### pooled_output will give the representation for [CLS]
### sequence_output will give representations for all tokens
## since it's classification task we will just use pooled_output
pooled_output, sequence_output = bert_model([input_ids_processed, input_mask_processed, segment_ids_processed])
#     bert_output = tf.keras.layers.Dropout(rate=bert_config.hidden_dropout_prob)(pooled_output) 

# This doesn't really matter as we will be loading the weights from saved checkpoint
initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config["initializer_range"])
#     output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(bert_output)
# dense_output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32,activation='softmax')(pooled_output)
dense_output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(pooled_output)
# log_probs = tf.nn.log_softmax(dense_output, axis=-1,name='scores')
probs = tf.nn.softmax(dense_output,axis=-1,name='scores')
# top_label_id = tf.argmax(log_probs,axis=-1)
top_label_id = tf.argmax(probs,axis=-1)
# sorted_indexes = tf.slice(tf.cast(tf.argsort(probs,axis=-1,direction='DESCENDING'),tf.int64),0,max_top_intents)
sorted_indexes = tf.cast(tf.argsort(probs,axis=-1,direction='DESCENDING'),tf.int64)[:,:max_top_intents]
# sorted_scores = tf.gather(log_probs,sorted_indexes, axis=-1)
# sorted_scores1 = tf.slice(tf.sort(probs,axis=-1,direction='DESCENDING',name="scores1"),0,max_top_intents)
sorted_scores1 = tf.sort(probs,axis=-1,direction='DESCENDING',name="scores1")[:,:max_top_intents]
# this identity operation was only required to get the name correctly as in tf2.3 the sort object's name was coming differently
# https://github.com/tensorflow/tensorflow/issues/39398
# https://stackoverflow.com/questions/38626424/how-to-assign-new-name-or-rename-an-existing-tensor-in-tensorflow
sorted_scores = tf.identity(sorted_scores1,name='scores_sorted')

top_label = LabelLookup(filepath=intent_file,default=default_label)(top_label_id)
labels = LabelLookup(filepath=intent_file,default=default_label,name="classes")(sorted_indexes)

loaded_model  = tf.keras.Model(inputs={'input_text':input_text},
                     outputs={"classes":labels,"scores":sorted_scores})


# loaded_model.compile()
#     checkpoint = tf.train.Checkpoint(model=model)
#     checkpoint.restore(checkpoint_path).assert_existing_objects_matched()
loaded_model.load_weights(checkpoint_path).assert_existing_objects_matched()


Two checkpoint references resolved to different objects (<tensorflow_hub.keras_layer.KerasLayer object at 0x7f7d85db3b50> and <tensorflow.python.keras.layers.core.Dense object at 0x7f7d85d36be0>).
Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<tensorflow_hub.keras_layer.KerasLayer object at 0x7f7d85db3b50> and <tensorflow.python.keras.layers.core.Dense object at 0x7f7d85d36be0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7f7d85d36be0> and <tensorflow.python.keras.engine.base_layer.TensorFlowOpLayer object at 0x7f7d6be87ac0>).
Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed 

In [506]:
sorted_scores1[:,:max_top_intents]

<tf.Tensor 'strided_slice_3:0' shape=(None, None) dtype=float32>

In [116]:
loaded_model.output_names = ['classes', 'scores']

In [117]:
loaded_model.output_names

ListWrapper(['classes', 'scores'])

In [118]:
output = loaded_model.predict(tf.constant(['talk to agent', 'remote not working']))
output

{'classes': array([[b'representative-request', b'other-other', b'get-help',
         b'channel_package-vague', b'payment_extension-request',
         b'channel_package-cancel', b'account-vague',
         b'reactivate-reinstate', b'return-query',
         b'comp_part_signal_loss-issue', b'channel_package-issue',
         b'spanish-query', b'channel_package-upgrade',
         b'billing-preferences', b'account-cancel',
         b'account-name_change', b'payment-query', b'channel-list',
         b'contract_expiry-query', b'autopay-cancel',
         b'blue_black_snowy-issue', b'account-address_change',
         b'channel_package-change', b'online_account-unable_login',
         b'appointment-tech_get_help', b'payment_arrangement-vague',
         b'dish_pause-cancel', b'contract-query',
         b'channel_package-query', b'payment-reinstate_service',
         b'autopay-setup', b'tech-issue', b'referral-how_to',
         b'dish_move-query', b'remote-setup', b'promotion-query',
         b'prom

In [119]:
sorted_scores1

<tf.Tensor 'TopKV2_1:0' shape=(None, None) dtype=float32>

In [120]:
sorted_scores

<tf.Tensor 'scores_sorted:0' shape=(None, None) dtype=float32>

In [121]:
loaded_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_text (InputLayer)         [(None,)]            0                                            
__________________________________________________________________________________________________
sentencepiece_tokenization (Sen ((None, None), (None 0           input_text[0][0]                 
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 17683968    sentencepiece_tokenization[0][0] 
                                                                 sentencepiece_tokenization[0][2] 
                                                                 sentencepiece_tokenization[0][1] 
_______________________________________________________________________________________

In [115]:
output['classes'][0]

array([b'representative-request', b'payment_extension-request',
       b'other-other', b'account-vague', b'billing-vague',
       b'appointment-schedule', b'dish_move-query',
       b'reactivate-reinstate', b'promotion-query',
       b'channel_package-vague', b'tech-issue', b'channel_package-change',
       b'account_info-get_help', b'dish_pause-query', b'get-help',
       b'appointment-reschedule', b'account-address_change',
       b'payment_arrangement-vague', b'internet-query', b'return-query',
       b'internet-get_help', b'comp_part_signal_loss-issue',
       b'return-get_help', b'equipment-upgrade', b'channel-list',
       b'account-transfer', b'blue_black_snowy-issue',
       b'online_account-password_reset', b'appointment-tech_get_help',
       b'appointment-query', b'equipment-query',
       b'channel_package-upgrade', b'remote-setup',
       b'payment_schedule-query', b'autopay-setup',
       b'channel_price-query', b'promotion-get_help', b'price-increase',
       b'equipment

In [76]:
label_map = {i:label for i,label in enumerate(intent_list)}

In [77]:
label_map.get(131)

'representative-request'

#### Let's put the loading model part into a function

In [512]:
def get_albert_model_saving(bert_config,input_meta_data,hub_url,checkpoint_path,
                            sentencepiece_path,label_file, max_top_intents=2):
    """This function return our classification model based on bert + updated bert part of model """

    max_seq_length = input_meta_data['max_seq_length']
    num_labels = input_meta_data['num_labels']
    default_label = tf.constant(input_meta_data['default_label'])
    #alternative way of giving input by specifying the entire text & do the preprocessing in the graph
    input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
    sentence_piece_layer = SentencepieceTokenization(model_path=sentencepiece_path,
                                                     max_seq_length=tf.constant(max_seq_length))
    input_ids_processed,segment_ids_processed,input_mask_processed = sentence_piece_layer(input_text)
    
    
    bert_model = hub.KerasLayer(hub_url,trainable=True,tags=None)
    ### pooled_output will give the representation for [CLS]
    ### sequence_output will give representations for all tokens
    ##since it's classification task we will just use pooled_output
    pooled_output, sequence_output = bert_model([input_ids_processed, input_mask_processed, segment_ids_processed])
    
    # This doesn't really matter as we will be loading the weights from saved checkpoint
    initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config["initializer_range"])
    # we don't want dropout for the model we are going to serve so skipping that
    dense_output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(pooled_output)
    # log_probs = tf.nn.softmax()
#     log_probs = tf.nn.log_softmax(dense_output, axis=-1)
    probs = tf.nn.softmax(dense_output, axis=-1)
    top_label_id = tf.argmax(probs,axis=-1)
    # sorted_scores = tf.gather(log_probs,sorted_indexes, axis=-1)
    sorted_indexes = tf.cast(tf.argsort(probs,axis=-1,direction='DESCENDING'),tf.int64)[:,:max_top_intents]
    # sorted_scores = tf.gather(log_probs,sorted_indexes, axis=-1)
    sorted_scores1 = tf.sort(probs,axis=-1,direction='DESCENDING',name="scores1")[:,:max_top_intents]
    
    # this identity operation was only required to get the name correctly as in tf2.3 the sort object's name was coming differently
    # https://github.com/tensorflow/tensorflow/issues/39398
    # https://stackoverflow.com/questions/38626424/how-to-assign-new-name-or-rename-an-existing-tensor-in-tensorflow
    # the identity operation also didn't help
    sorted_scores = tf.identity(sorted_scores1,name='scores_sorted')
    top_label = LabelLookup(filepath=intent_file,default=default_label)(top_label_id)
    labels = LabelLookup(filepath=intent_file,default=default_label,name="classes")(sorted_indexes)
    # in the new version the name of the variable seems to be taken rather than the key of the dictionary
    model  = tf.keras.Model(inputs={'input_text':input_text},
                         outputs={"classes":labels,"scores":sorted_scores})
    # FIXME the following is a hack (not sure if it will create other issues)
    model.output_names = ['classes', 'scores']

#     checkpoint = tf.train.Checkpoint(model=model)
#     checkpoint.restore(checkpoint_path).assert_existing_objects_matched()
    model.load_weights(checkpoint_path).assert_existing_objects_matched()
    return model, bert_model, input_text, labels, sorted_scores

In [513]:
tf.keras.backend.clear_session()

In [514]:
classification_model_to_save, bert_updated,input_text,labels, sorted_log_probs = get_albert_model_saving(bert_config,
                                                                     input_meta_data,
                                                                     bert_hub_url,
                                                                     checkpoint_path=tf.train.latest_checkpoint(output_folder),
                                                                    sentencepiece_path=sentencepiece_path,
                                                                     label_file=intent_file
                                                                    )


Two checkpoint references resolved to different objects (<tensorflow_hub.keras_layer.KerasLayer object at 0x7f7ecb941910> and <tensorflow.python.keras.layers.core.Dense object at 0x7f7d6be3ed30>).
Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<tensorflow_hub.keras_layer.KerasLayer object at 0x7f7ecb941910> and <tensorflow.python.keras.layers.core.Dense object at 0x7f7d6be3ed30>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7f7d6be3ed30> and <tensorflow.python.keras.engine.base_layer.TensorFlowOpLayer object at 0x7f7d6a1eba00>).
Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed 

In [515]:
classification_model_to_save.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_text (InputLayer)         [(None,)]            0                                            
__________________________________________________________________________________________________
sentencepiece_tokenization (Sen ((None, 128), (None, 0           input_text[0][0]                 
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 17683968    sentencepiece_tokenization[0][0] 
                                                                 sentencepiece_tokenization[0][2] 
                                                                 sentencepiece_tokenization[0][1] 
_______________________________________________________________________________________

In [521]:
output = classification_model_to_save.predict(['','hello','talk to agent', 'remote not working'])
output

{'classes': array([[b'other-other', b'payment-status'],
        [b'greet-hello', b'yes-give'],
        [b'representative-request', b'other-other'],
        [b'remote-setup', b'remote-not_working']], dtype=object),
 'scores': array([[0.12820116, 0.04802238],
        [0.8430833 , 0.02728763],
        [0.9865248 , 0.00239532],
        [0.99100107, 0.00308522]], dtype=float32)}

In [522]:
output['classes'][:,0]

array([b'other-other', b'greet-hello', b'representative-request',
       b'remote-setup'], dtype=object)

In [519]:
# final_model_path=os.path.join(output_folder,'final_model')
final_model_path='/space/engineering/tf_serve/models/dish_albert'
final_versioned_model_path = os.path.join(final_model_path,'4')
classification_model_to_save.save(final_versioned_model_path)

INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/dish_albert/4/assets
Assets written to: /space/engineering/tf_serve/models/dish_albert/4/assets


### loading the saved model with preprocessing & trying it out

In [527]:
loaded_final_model = tf.saved_model.load(final_versioned_model_path)

In [531]:
serving_default = loaded_final_model.signatures['serving_default']
serving_default

<ConcreteFunction signature_wrapper(input_text) at 0x7F7D6172E1C0>

In [534]:
serving_default(tf.constant(['','hello','talk to agent', 'remote not working']))

{'scores': <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[0.12820116, 0.04802238],
        [0.8430833 , 0.02728763],
        [0.9865248 , 0.00239532],
        [0.99100107, 0.00308522]], dtype=float32)>,
 'classes': <tf.Tensor: shape=(4, 2), dtype=string, numpy=
 array([[b'other-other', b'payment-status'],
        [b'greet-hello', b'yes-give'],
        [b'representative-request', b'other-other'],
        [b'remote-setup', b'remote-not_working']], dtype=object)>}

### running against a held out test set
ideally we need a held out test set, but using the validation set again just to have the testing code here

In [535]:
df_test = df_val.copy()

In [140]:
df_val.head()

,filename,text,granular_intent,ru_intent
20421,a9cf6bee-3971-4516-459f-2243d9a606fd,I want to inform you that I don't have electri...,billing-issue,billing-issue
22946,0f050cf9-88d7-44bd-23c8-5f26674f5b6a,What happen to my bill,billing-query,billing-query
8207,INT-sv1appis11-1503849377195-243917,I need to replace my remote control.,remote-place_order,remote-place_order
14144,INT-sv1appis14-1504137880316-305603_1158,Reciever is saying not authorized on every cha...,receiver_issue-get_help,receiver_issue-get_help
8164,6e1842f4-63a1-4259-3999-6928033ce7eb,i need to update my credit card info,payment-get_help,payment-get_help


In [206]:
output = classification_model_to_save.predict(df_val['text'].array[:20].to_numpy())
output

{'classes': array([[b'billing-issue', b'price-increase', b'billing-vague', ...,
         b'app-query', b'dvr-query', b'receiver_upgrade-query'],
        [b'billing-issue', b'billing-vague', b'billing-query', ...,
         b'ppv-cancel', b'order-query', b'receiver_upgrade-query'],
        [b'remote-place_order', b'remote-order_status',
         b'remote-not_working', ..., b'get-help', b'billing-vague',
         b'other-other'],
        ...,
        [b'channel_package-issue', b'streaming-issues',
         b'comp_part_signal_loss-issue', ..., b'order-cancel',
         b'internet-price_query', b'appointment-cancel'],
        [b'price-increase', b'billing-issue',
         b'account_cancel-promotion_seek', ..., b'online_account-setup',
         b'dish_pause-cancel', b'billing-preferences'],
        [b'greet-hello', b'yes-give', b'no-give', ..., b'promotion-query',
         b'billing-preferences', b'account-address_change']], dtype=object),
 'scores': array([[9.90572035e-01, 1.36903033e-03, 7

In [536]:
def get_predicted_intent_per_row(row, classification_model, text_column='text',out_of_domain_intent='none_none'):
    text = row['text']
    try:
        output = classification_model.predict([text])
        # 1st index is for batch & batch size is 1, 2nd is the list of intents, taking the top intent
        row['predicted_intent'] = output['classes'][0][0]
        row['score'] = output['scores'][0][0]
    except Exception as e:
        print(f'exception occured, text {text} classifying as out of domain intent error {e}')
        row['predicted_intent']  = out_of_domain_intent
        row['score'] = 1
    return row

In [537]:
print(f'size of test data {len(df_test)}')
df_test = df_test.apply(lambda row:get_predicted_intent_per_row(row,classification_model_to_save,text_column='text'),axis=1)
df_test.head()

size of test data 2572


,filename,text,granular_intent,ru_intent,predicted_intent,score
20421,a9cf6bee-3971-4516-459f-2243d9a606fd,I want to inform you that I don't have electri...,billing-issue,billing-issue,b'billing-issue',0.990572
22946,0f050cf9-88d7-44bd-23c8-5f26674f5b6a,What happen to my bill,billing-query,billing-query,b'billing-issue',0.983889
8207,INT-sv1appis11-1503849377195-243917,I need to replace my remote control.,remote-place_order,remote-place_order,b'remote-place_order',0.939638
14144,INT-sv1appis14-1504137880316-305603_1158,Reciever is saying not authorized on every cha...,receiver_issue-get_help,receiver_issue-get_help,b'receiver_issue-get_help',0.598178
8164,6e1842f4-63a1-4259-3999-6928033ce7eb,i need to update my credit card info,payment-get_help,payment-get_help,b'payment-get_help',0.947868


In [94]:
serving_default.structured_input_signature

((),
 {'input_text': TensorSpec(shape=(None,), dtype=tf.string, name='input_text')})

In [95]:
serving_default.structured_outputs

{'label_lookup': TensorSpec(shape=(None,), dtype=tf.string, name='label_lookup')}

In [96]:
serving_default(tf.constant(['talk to an agent']))

{'label_lookup': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'other-other'], dtype=object)>}